In [1]:
#diretorio_datasets = "/home/caioabreu/workspace/tcc/datasets/municipio"
diretorio_datasets = "/home/matteus-paula/Documents/dados/dados_tcc/municipio"

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

> ## Dados Municipio:<br>
- Base de Municipios para ligar codigo ibge da tabela escolas do inep (6 digitos) com investimentos (7 digitos)
- Fonte: Municipio dados territoriais
- http://www2.datasus.gov.br/DATASUS/index.php?area=060206
- ftp://ftp.datasus.gov.br/territorio/tabelas


In [3]:
municipios = pd.read_csv(f'{diretorio_datasets}/tb_municip.csv', 
                            delimiter=";", encoding = "ISO-8859-1",
                        names=["CO_MUNICIP","CO_MUNICDV","CO_STATUS","CO_TIPO",
                               "DS_NOME","DS_NOMEPAD","DS_OBSERV","CO_ALTER",
                               "CO_ALTERDV","CO_REGIAO","CO_UF","IN_CAPITAL",
                               "IN_AMAZLEG","IN_SEMIAR","IN_FRONTZN","IN_FRONTFX",
                               "IN_POBREZA","DT_INSTAL","DT_EXTIN","CO_SUCESS",
                               "NU_ORDEM","NU_ORDMAP","NU_LATITUD","NU_LONGIT","NU_ALTITUD","NU_AREA"])

In [4]:
municipios.head()

,CO_MUNICIP,CO_MUNICDV,CO_STATUS,CO_TIPO,DS_NOME,DS_NOMEPAD,DS_OBSERV,CO_ALTER,CO_ALTERDV,CO_REGIAO,...,IN_POBREZA,DT_INSTAL,DT_EXTIN,CO_SUCESS,NU_ORDEM,NU_ORDMAP,NU_LATITUD,NU_LONGIT,NU_ALTITUD,NU_AREA
0,0,0,IGNOR,IGNOR,Ignorado ou exterior,IGNORADO OU EXTERIOR,Ignorado,", ,000001-009999,999999",", ,0000001-0099999,9999999",0,...,,NaN,NaN,NaN,8,,"0,00000","0,00000",0,"0,000"
1,110000,1100000,IGNOR,IGNOR,Município ignorado - RO,MUNICIPIO IGNORADO - RO,NaN,",119999",",1199999",1,...,,NaN,NaN,NaN,8,,"0,00000","0,00000",0,"0,000"
2,110001,1100015,ATIVO,MUNIC,Alta Floresta D'Oeste,ALTA FLORESTA D'OESTE,NaN,NaN,NaN,1,...,N,1986.0,NaN,NaN,1,1,"-11,93554","-61,99982",338,"7067,025"
3,110002,1100023,ATIVO,MUNIC,Ariquemes,ARIQUEMES,NaN,NaN,NaN,1,...,N,1977.0,NaN,NaN,1,1,"-9,90846","-63,03327",139,"4426,571"
4,110003,1100031,ATIVO,MUNIC,Cabixi,CABIXI,NaN,NaN,NaN,1,...,N,1989.0,NaN,NaN,1,1,"-13,49976","-60,54431",236,"1314,352"


In [5]:
municipios_final = municipios[["CO_MUNICIP","CO_MUNICDV", "DS_NOME"]][municipios['CO_STATUS'] == 'ATIVO']
municipios_final.head()

,CO_MUNICIP,CO_MUNICDV,DS_NOME
2,110001,1100015,Alta Floresta D'Oeste
3,110002,1100023,Ariquemes
4,110003,1100031,Cabixi
5,110004,1100049,Cacoal
6,110005,1100056,Cerejeiras


In [6]:
 municipios_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 2 to 5658
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CO_MUNICIP  5570 non-null   int64 
 1   CO_MUNICDV  5570 non-null   int64 
 2   DS_NOME     5570 non-null   object
dtypes: int64(2), object(1)
memory usage: 174.1+ KB


In [7]:
municipios_final['SIZE_OF_CO_MUNICIP'] = municipios_final['CO_MUNICIP'].apply(lambda x: len(str(x)))
municipios_final['SIZE_OF_CO_MUNICDV'] = municipios_final['CO_MUNICDV'].apply(lambda x: len(str(x)))

In [8]:
municipios_final['CO_MUNICIP'][(municipios_final['SIZE_OF_CO_MUNICIP'] != 6)].count()

0

In [9]:
municipios_final['CO_MUNICDV'][(municipios_final['SIZE_OF_CO_MUNICDV'] != 7)].count()

0

In [10]:
municipios_final.isnull().any()

CO_MUNICIP            False
CO_MUNICDV            False
DS_NOME               False
SIZE_OF_CO_MUNICIP    False
SIZE_OF_CO_MUNICDV    False
dtype: bool

> ## Dados Estatisticas Municipio:
- Base populacao estimada
- https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html
- ftp://ftp.ibge.gov.br/Estimativas_de_Populacao/Estimativas_2017/

In [11]:
municipios_est_pop = pd.read_excel(f'{diretorio_datasets}/estimativa_TCU_2017_20190919.xls', 
                                   header=None, sheet_name='Municípios')

In [12]:
municipios_est_pop.head()

,0,1,2,3,4
0,ESTIMATIVAS DA POPULAÇÃO RESIDENTE NOS MUNICÍP...,NaN,NaN,NaN,NaN
1,UF,COD. UF,COD. MUNIC,NOME DO MUNICÍPIO,POPULAÇÃO ESTIMADA
2,RO,11,00015,Alta Floresta D'Oeste,25437
3,RO,11,00023,Ariquemes,107345
4,RO,11,00031,Cabixi,6224


- #### Função para transformar os COD_UF e COD_MUN em string e concatena-los para formar o codigo ibge do municipio com 7 digitos (dados estimativa_TCU)

In [15]:
def transform_mun_pop(input_df):

    dfs = input_df.iloc[2:]
    dfs.columns = ['UF', 'COD_UF', 'COD_MUN', 'NM_MUNICIPIO', 'NUM_POPULACAO']
    
    f = lambda x, y : str(x).strip() + str(y).strip()
    dfs['COD_MUNICIPIO'] = dfs[['COD_UF', 'COD_MUN']].apply(lambda x: f(*x), axis=1)
    
    dfs.drop(dfs.tail(16).index,inplace=True)
    
    return dfs

In [16]:
municipios_est_pop_df = transform_mun_pop(municipios_est_pop)
municipios_est_pop_df.head()

/home/matteus-paula/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/matteus-paula/.local/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,UF,COD_UF,COD_MUN,NM_MUNICIPIO,NUM_POPULACAO,COD_MUNICIPIO
2,RO,11,00015,Alta Floresta D'Oeste,25437,1100015
3,RO,11,00023,Ariquemes,107345,1100023
4,RO,11,00031,Cabixi,6224,1100031
5,RO,11,00049,Cacoal,88507,1100049
6,RO,11,00056,Cerejeiras,17934,1100056


- #### Transformando o COD_MUNICIPIO em integer para depois fazer ligação com tabela de escolas que já é numerico

In [17]:
municipios_est_pop_df['COD_MUNICIPIO'] = municipios_est_pop_df['COD_MUNICIPIO'].astype(int)

/home/matteus-paula/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


<br>

- #### Transformando o NUM_POPULACAO em string para corrigir numeros que vieram com parenteses e espaços ex: 654564 (1)

In [18]:
municipios_est_pop_df['NUM_POPULACAO'] = municipios_est_pop_df['NUM_POPULACAO'].astype(str)
municipios_est_pop_df['NUM_POPULACAO'] = municipios_est_pop_df['NUM_POPULACAO'].str.\
    replace(r'\.|[^\d]*\(\d+\)$', '').astype(int)
municipios_est_pop_df.head()

/home/matteus-paula/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/matteus-paula/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,UF,COD_UF,COD_MUN,NM_MUNICIPIO,NUM_POPULACAO,COD_MUNICIPIO
2,RO,11,00015,Alta Floresta D'Oeste,25437,1100015
3,RO,11,00023,Ariquemes,107345,1100023
4,RO,11,00031,Cabixi,6224,1100031
5,RO,11,00049,Cacoal,88507,1100049
6,RO,11,00056,Cerejeiras,17934,1100056


In [20]:
municipios_est_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 2 to 5571
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UF             5570 non-null   object
 1   COD_UF         5570 non-null   object
 2   COD_MUN        5570 non-null   object
 3   NM_MUNICIPIO   5570 non-null   object
 4   NUM_POPULACAO  5570 non-null   int64 
 5   COD_MUNICIPIO  5570 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 304.6+ KB


In [21]:
municipios_est_pop_df.isnull().any()

UF               False
COD_UF           False
COD_MUN          False
NM_MUNICIPIO     False
NUM_POPULACAO    False
COD_MUNICIPIO    False
dtype: bool

<br>

In [22]:
municipios_pop = pd.merge(municipios_est_pop_df, municipios_final, left_on='COD_MUNICIPIO', right_on='CO_MUNICDV', how='inner')
municipios_pop.head()

,UF,COD_UF,COD_MUN,NM_MUNICIPIO,NUM_POPULACAO,COD_MUNICIPIO,CO_MUNICIP,CO_MUNICDV,DS_NOME,SIZE_OF_CO_MUNICIP,SIZE_OF_CO_MUNICDV
0,RO,11,00015,Alta Floresta D'Oeste,25437,1100015,110001,1100015,Alta Floresta D'Oeste,6,7
1,RO,11,00023,Ariquemes,107345,1100023,110002,1100023,Ariquemes,6,7
2,RO,11,00031,Cabixi,6224,1100031,110003,1100031,Cabixi,6,7
3,RO,11,00049,Cacoal,88507,1100049,110004,1100049,Cacoal,6,7
4,RO,11,00056,Cerejeiras,17934,1100056,110005,1100056,Cerejeiras,6,7


In [23]:
municipios_pop['CO_MUNICIPIO_IBGE'] = municipios_pop['CO_MUNICIP']
municipios_pop['CO_MUNICIPIO'] = municipios_pop['COD_MUNICIPIO']

In [24]:
municipios_final = municipios_pop[['UF', 'NM_MUNICIPIO', 'NUM_POPULACAO', 'CO_MUNICIPIO_IBGE', 'CO_MUNICIPIO']]
municipios_final.head()

,UF,NM_MUNICIPIO,NUM_POPULACAO,CO_MUNICIPIO_IBGE,CO_MUNICIPIO
0,RO,Alta Floresta D'Oeste,25437,110001,1100015
1,RO,Ariquemes,107345,110002,1100023
2,RO,Cabixi,6224,110003,1100031
3,RO,Cacoal,88507,110004,1100049
4,RO,Cerejeiras,17934,110005,1100056


In [26]:
municipios_final.to_csv(f'{diretorio_datasets}/municipio.csv', sep=';', encoding='utf-8', index=False)